Wczytanie i przetworzenie zbiorów danych

In [11]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas_profiling

In [2]:
nazwy_kolumn = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16']
dane = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', names = nazwy_kolumn)

In [3]:
pandas_profiling.ProfileReport(dane)

Number of variables,16
Number of observations,690
Total Missing (%),0.0%
Total size in memory,86.3 KiB
Average record size in memory,128.1 B
Numeric,4
Categorical,12
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [4]:
for i in nazwy_kolumn:
    dane[i]=dane[i].replace('?', -1)


In [5]:
X = dane.drop('A16', axis = 1)
y = dane['A16']

y = y.factorize()[0]
X['A1'] = X['A1'].factorize()[0]
X['A4'] = X['A4'].factorize()[0]
X['A5'] = X['A5'].factorize()[0]
X['A6'] = X['A6'].factorize()[0]
X['A7'] = X['A7'].factorize()[0]
X['A9'] = X['A9'].factorize()[0]
X['A10'] = X['A10'].factorize()[0]
X['A12'] = X['A12'].factorize()[0]
X['A13'] = X['A13'].factorize()[0]
X['A14'] = X['A14'].factorize()[0]

In [6]:
X.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,0,30.83,0.000,0,0,0,0,1.25,0,0,1,0,0,0,0
1,1,58.67,4.460,0,0,1,1,3.04,0,0,6,0,0,1,560
2,1,24.50,0.500,0,0,1,1,1.50,0,1,0,0,0,2,824
3,0,27.83,1.540,0,0,0,0,3.75,0,0,5,1,0,3,3
4,0,20.17,5.625,0,0,0,0,1.71,0,1,0,0,1,4,0


Trenowanie drzewa na pełnym zbiorze danych

In [9]:
drzewo = DecisionTreeClassifier()
drzewo.fit(X=X, y=y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [10]:
drzewo.score(X=X, y=y)

0.8608695652173913

Trenowanie drzewa na zbiorach uczącym i testowym

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

In [15]:
drzewo = DecisionTreeClassifier()
drzewo.fit(X=X_train, y=y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [16]:
drzewo.score(X=X_test, y=y_test)

0.7463768115942029

Wybór najlepszych hiperparametrów drzewa